In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('/content/E-commerce Website Logs.csv', on_bad_lines='skip')

<ipython-input-2-2648153816>:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/E-commerce Website Logs.csv', on_bad_lines='skip')


In [3]:
df.head()

,accessed_date,duration_(secs),network_protocol,ip,bytes,accessed_Ffom,age,gender,country,membership,language,sales,returned,returned_amount,pay_method
0,2017-03-14 17:43:57.172,2533,TCP,1.10.195.126,20100,Chrome,28,Female,CA,Normal,English,261.9600,No,0.0,Credit Card
1,2017-03-14 17:43:57.172,4034,TCP,1.1.217.211,20500,Mozilla Firefox,21,Male,AR,Normal,English,731.9400,No,0.0,Debit Card
2,2017-03-14 17:43:26.135,1525,TCP,1.115.198.107,90100,Mozilla Firefox,20,Male,PL,Normal,English,14.6200,No,0.0,Cash
3,2017-03-14 17:43:26.135,4572,TCP,1.121.152.143,100300,Mozilla Firefox,66,Female,IN,Normal,Spanish,957.5775,No,0.0,Credit Card
4,2017-03-14 18:17:09.005,3652,TCP,1.123.135.213,270200,Mozilla Firefox,53,Female,KR,Normal,Spanish,22.3680,No,0.0,Cash


In [4]:
# Check for missing values and data types
missing_values = df.isnull().sum()
data_types = df.dtypes

# Check unique values in each column to detect anomalies or inconsistencies
unique_values_summary = df.nunique()

# View sample values from 'age' and 'returned' to evaluate cleaning needs
age_sample = df['age'].unique()
returned_sample = df['returned'].unique()

missing_values, data_types, unique_values_summary, age_sample, returned_sample

(accessed_date           0
 duration_(secs)         0
 network_protocol        0
 ip                      0
 bytes                   0
 accessed_Ffom           0
 age                 73381
 gender                  0
 country                 0
 membership              0
 language                0
 sales                   0
 returned                0
 returned_amount         0
 pay_method              0
 dtype: int64,
 accessed_date        object
 duration_(secs)       int64
 network_protocol     object
 ip                   object
 bytes                 int64
 accessed_Ffom        object
 age                  object
 gender               object
 country              object
 membership           object
 language             object
 sales               float64
 returned             object
 returned_amount     float64
 pay_method           object
 dtype: object,
 accessed_date        81747
 duration_(secs)       3501
 network_protocol         5
 ip                  137199
 bytes           

In [6]:
# Remove rows with missing or invalid age and 'Normal' in age column
df_cleaned = df[~df['age'].isin(['--', 'Normal'])]
df_cleaned = df_cleaned.dropna(subset=['age'])

# Convert 'age' to integer
df_cleaned['age'] = df_cleaned['age'].astype(int)

# Convert 'accessed_date' to datetime
df_cleaned['accessed_date'] = pd.to_datetime(df_cleaned['accessed_date'], errors='coerce')

# Remove rows with invalid dates
df_cleaned = df_cleaned.dropna(subset=['accessed_date'])

# The 'sales' and 'bytes' columns also seem to have mixed types.
# Convert 'sales' and 'bytes' to numeric, coercing errors
df_cleaned['sales'] = pd.to_numeric(df_cleaned['sales'], errors='coerce')
df_cleaned['bytes'] = pd.to_numeric(df_cleaned['bytes'], errors='coerce')

# Remove rows where 'sales' or 'bytes' could not be converted to numeric
df_cleaned = df_cleaned.dropna(subset=['sales', 'bytes'])


# Create new calculated fields
df_cleaned['net_sales'] = df_cleaned['sales'] - df_cleaned['returned_amount']
df_cleaned['is_converted'] = df_cleaned['sales'].apply(lambda x: 1 if x > 0 else 0)
# Fix for 'returned' column: Ensure it is treated as string before comparison
df_cleaned['is_returned'] = df_cleaned['returned'].astype(str).apply(lambda x: 1 if x == 'Yes' else 0)

# Save the cleaned dataset to CSV for Tableau import
# cleaned_file_path = "/mnt/data/cleaned_ecommerce_logs.csv" # This path is for Colab Enterprise and won't work in standard Colab
# Using a standard /content path for saving in Colab
cleaned_file_path = "/content/cleaned_ecommerce_logs.csv"
df_cleaned.to_csv(cleaned_file_path, index=False)

# Show cleaned dataset shape and new columns
df_cleaned.shape, df_cleaned[['age', 'accessed_date', 'net_sales', 'is_converted', 'is_returned']].head()

((84714, 18),
    age           accessed_date  net_sales  is_converted  is_returned
 0   28 2017-03-14 17:43:57.172   261.9600             1            0
 1   21 2017-03-14 17:43:57.172   731.9400             1            0
 2   20 2017-03-14 17:43:26.135    14.6200             1            0
 3   66 2017-03-14 17:43:26.135   957.5775             1            0
 4   53 2017-03-14 18:17:09.005    22.3680             1            0)